In [2]:
import json, os, subprocess
import tqdm
from time import time
import pandas as pd
from neuralNetwork import NeuralNetwork, In_between_epochs
import torch.nn as nn
import torch
from sklearn.metrics import f1_score, accuracy_score
from helper import get_dataloader

In [3]:
class Model(NeuralNetwork):
    def __init__(self, input_shape, output_shape) -> None:
        super().__init__()
        self.linear_0 = nn.Linear(input_shape, input_shape * 4)
        self.linear_1 = nn.Linear(input_shape * 4, input_shape * 3)
        self.linear_2 = nn.Linear(input_shape * 3, input_shape * 2)
        self.linear_3 = nn.Linear(input_shape * 2, input_shape * 1)
        self.linear_4 = nn.Linear(input_shape, output_shape * 10)
        self.linear_5 = nn.Linear(output_shape * 10, output_shape * 5)
        self.linear_6 = nn.Linear(output_shape * 5, output_shape * 3)
        self.output = nn.Linear(output_shape * 3, output_shape)

        self.relu = nn.ReLU()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        out = self.linear_0(input)
        out = self.relu(out)
        out = self.linear_1(out) 
        out = self.relu(out)
        out = self.linear_2(out)
        out = self.relu(out)
        out = self.linear_3(out)
        out = self.relu(out)
        out = self.linear_4(out)
        out = self.relu(out)

        out = self.linear_5(out)
        out = self.relu(out)
        out = self.linear_6(out)
        out = self.relu(out)
        out = self.output(out)
        return out
    
class EarlyStopping(In_between_epochs):
    def __init__(self, delta, patience):
        self.delta = delta
        self.patience = patience
        self.current_patience = 0
        self.best_valid_loss = 100000000000000000000
        self.best_model = None
        self.epochs = 0
    
    def __call__(self, model:torch.nn.Module, loaders:dict[str,torch.utils.data.DataLoader], device:'torch.device|str', output_extraction_function, losses:dict[str, float]) -> bool:
        self.epochs += 1
        if losses["validation"] < self.best_valid_loss - self.delta:
            self.best_valid_loss = losses["validation"]
            if self.best_model is None:
                self.best_model = model
            self.best_model.load_state_dict(model.state_dict())
            self.current_patience = 0
        else:
            self.current_patience += 1
            if self.current_patience >= self.patience:
                return True
        return False
    def reset(self):
        self.current_patience = 0
        self.epochs = 0

In [7]:
def train(model, train, validation, min_lr, start_lr, early_stopping, frac, device):
    tot_train_data, tot_val_data = [], []
    lr = start_lr
    while lr > min_lr:
        train_data, validation_data = model.train_network(train, 
                        validation, 
                        torch.optim.Adam, 
                        loss_function=nn.CrossEntropyLoss(),
                        device=device, 
                        batch_size=32,
                        verbose=False, 
                        output_extraction_function= lambda x: torch.max(x, -1)[1].view(-1).cpu(), 
                        metrics={
                        "accuracy": accuracy_score, 
                        "f1_score": lambda y_true, y_pred: f1_score(y_true, y_pred, average="macro")},
                        in_between_epochs = {"early_stopping": early_stopping},
                        learning_rate=lr,
                        epochs=30)
        train_data["epochs"] = early_stopping.epochs
        train_data["lr"] = lr
        validation_data["epochs"] = early_stopping.epochs
        validation_data["lr"] = lr
        tot_train_data.append(train_data)
        tot_val_data.append(validation_data)
        model.load_state_dict(early_stopping.best_model.state_dict())
        lr = lr * frac
        early_stopping.reset()

In [15]:
f = open("../data/datasets/dataset_CoveringArray-2024-05-09.json")
dataset = json.load(f)
f.close()

In [17]:
folds = [0, 1, 2]
models = []
for fold in folds:
    df = pd.read_csv(f"../data/csv_features/features_covering_array_fold_{fold}.csv")
    combinations = [t["combination"] for t in dataset[0]["all_times"]]
    x = []
    y = []
    for datapoint in tqdm.tqdm(dataset, "dataset creation"):
        instance = datapoint["instance_name"]
        features = df[df["inst"] == instance].to_dict()
        idx = list(features["inst"].keys())[0]
        features = {key: features[key][idx] for key in features.keys()}
        keys = list(features.keys())
        keys.pop(keys.index("inst"))
        keys.pop(keys.index("time"))
        features = [features[key] for key in keys]
        x.append(torch.tensor(features))
        correct_result = torch.zeros(len(combinations))
        correct_result[combinations.index(datapoint["combination"])] = 1.
        y.append(correct_result)
    train_dataloader, validation_dataloader, test_dataloader = get_dataloader(x, y, 32, [fold])
    input_shape = len(x[0])
    output_shape = len(y[0])
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Model(input_shape, output_shape)
    early_stopping = EarlyStopping(.001, 5)
    train(model, train_dataloader, validation_dataloader, 5e-7, 1e-3, early_stopping, .6, device)
    best_model = model
    best_model.load_state_dict(early_stopping.best_model.state_dict())
    models.append(best_model)
    best_model.eval()
    best_model.to(device)
    trues = []
    preds = []
    e = lambda x: torch.max(x, -1)[1].view(-1).cpu()
    with torch.no_grad():
        for _, (current_x, current_y) in enumerate(validation_dataloader):
            current_x = current_x.to(device)
            y_pred = best_model(current_x)
            preds += e(y_pred)
            trues += e(current_y)
            del y_pred
            del current_x
    print(f"final accuracy for validation (fold {fold}): {accuracy_score(trues, preds)}")

dataset creation: 100%|██████████| 2236/2236 [01:00<00:00, 37.00it/s]


final accuracy for validation (fold 0): 0.7205882352941176


dataset creation: 100%|██████████| 2236/2236 [00:59<00:00, 37.41it/s]


final accuracy for validation (fold 1): 0.6911764705882353


dataset creation: 100%|██████████| 2236/2236 [00:58<00:00, 38.36it/s]


final accuracy for validation (fold 2): 0.75


In [24]:
model = model.to(device)
model.eval()

Model(
  (linear_0): Linear(in_features=772, out_features=3088, bias=True)
  (linear_1): Linear(in_features=3088, out_features=2316, bias=True)
  (linear_2): Linear(in_features=2316, out_features=1544, bias=True)
  (linear_3): Linear(in_features=1544, out_features=772, bias=True)
  (linear_4): Linear(in_features=772, out_features=40, bias=True)
  (linear_5): Linear(in_features=40, out_features=20, bias=True)
  (linear_6): Linear(in_features=20, out_features=12, bias=True)
  (output): Linear(in_features=12, out_features=4, bias=True)
  (relu): ReLU()
)

In [8]:
def get_data(data, test_buckets = []):
    BUCKETS = 10

    N_ELEMENTS = len(data)

    BUCKET_SIZE = N_ELEMENTS // BUCKETS

    local = data.copy()
    test = []
    
    for bucket in test_buckets:
        idx = bucket * BUCKET_SIZE
        for _ in range(BUCKET_SIZE):
            test.append(local.pop(idx))

    train_elements = (len(local) // 10) * 9
    train = local[:train_elements]

    validation = local[train_elements:]

    
    return train, validation, test

In [20]:
def test(dataset, model, combinations, desc):
    tot_time = 0
    combs = {comb:0 for comb in combinations}
    trues = []
    preds = []
    for datapoint in tqdm.tqdm(dataset, desc):
        instance = datapoint["instance_name"]
        features = df[df["inst"] == instance].to_dict()
        idx = list(features["inst"].keys())[0]
        features = {key: features[key][idx] for key in features.keys()}
        keys = list(features.keys())
        keys.pop(keys.index("inst"))
        keys.pop(keys.index("time"))
        features = [features[key] for key in keys]
        x = torch.tensor(features)
        x = x.to(device)
        y_pred = model(x)
        del x
        y_pred = int(torch.max(y_pred, -1)[1].view(-1).cpu())
        comb = combinations[y_pred]
        times = {t["combination"]: t["time"] for t in datapoint["all_times"]}
        tot_time += times[comb]
        for key in times:
            combs[key] += times[key]
        trues.append(combinations.index(datapoint["combination"]))
        preds.append(y_pred)
    print(f"accuracy: {accuracy_score(trues, preds):2f}")
    return tot_time, combs

In [21]:
folds_data = {}
for fold in [0,1,2]:
    train_data, validaion_data, test_data = get_data(dataset, [fold])
    model = models[fold]
    with torch.no_grad():
        tot_time_train, combs_train = test(train_data, model, combinations, f"train fold: {fold}")
        tot_time_val, combs_val = test(validaion_data, model, combinations, f"validation fold: {fold}")
        tot_time_test, combs_test = test(test_data, model, combinations, f"test fold: {fold}")
        folds_data[fold] = {"train": {"time": tot_time_train, "combs": combs_train}, "validation": {"time": tot_time_val, "combs": combs_val}, "test": {"time": tot_time_test, "combs": combs_test}}

train fold: 0: 100%|██████████| 1809/1809 [00:54<00:00, 33.49it/s]


accuracy: 0.559978


validation fold: 0: 100%|██████████| 204/204 [00:06<00:00, 31.95it/s]


accuracy: 0.588235


test fold: 0: 100%|██████████| 223/223 [00:06<00:00, 32.44it/s]


...

accuracy: 0.800442


validation fold: 2: 100%|██████████| 204/204 [00:06<00:00, 31.82it/s]


accuracy: 0.750000


test fold: 2: 100%|██████████| 223/223 [00:06<00:00, 35.42it/s]

accuracy: 0.744395


In [22]:
for fold in [0, 1, 2]:
    data = folds_data[fold]
    for dataset in ["train", "validation", "test"]:
        dataset_data = data[dataset]
        combs = dataset_data["combs"]
        tot_time = dataset_data["time"]
        sb_model, sb_time = min(combs.items(), key = lambda x: x[1])
        print(f"""[fold {fold} - {dataset}]
sb combination: {sb_model}
sb time: {sb_time:,.2f}
pred time: {tot_time:,.2f}
pred/sb:{tot_time/sb_time:,.2f}
-------------------------------""")
    print("====================================")

[fold 0 - train]
sb combination: kissat_01_compact.eprime
sb time: 34,669.20
pred time: 403,799.53
pred/sb:11.65
-------------------------------
[fold 0 - validation]
sb combination: kissat_01_compact.eprime
sb time: 4,888.05
pred time: 76,120.45
pred/sb:15.57
-------------------------------
[fold 0 - test]
sb combination: kissat_01_compact.eprime
sb time: 4,288.49
pred time: 7,757.74
pred/sb:1.81
-------------------------------
[fold 1 - train]
sb combination: kissat_01_compact.eprime
sb time: 33,382.17
pred time: 331,066.31
pred/sb:9.92
-------------------------------
[fold 1 - validation]
sb combination: kissat_01_compact.eprime
sb time: 4,888.05
pred time: 76,124.49
pred/sb:15.57
-------------------------------
[fold 1 - test]
sb combination: kissat_01_compact.eprime
sb time: 5,575.52
pred time: 77,383.14
pred/sb:13.88
-------------------------------
[fold 2 - train]
sb combination: kissat_01_compact.eprime
sb time: 33,907.66
pred time: 97,527.99
pred/sb:2.88
----------------------

In [ ]:
31799.43999999997
33905.89000000003
43845.739999999976

In [63]:
vb = 0
for datapoint in tqdm.tqdm(dataset):
    vb += datapoint["time"]
print(vb)

100%|██████████| 2236/2236 [00:00<00:00, 5036768.93it/s]

31799.43999999997
